In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-04 13:00:00+00:00


In [4]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [30]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=2
)

query = fg.select_all()

2025-03-04 07:50:00,754 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 07:50:00,756 INFO: Initializing external client
2025-03-04 07:50:00,756 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 07:50:01,983 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215646


In [32]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (8.73s) 


In [35]:
df

,pickup_hour,pickup_location_id,rides
0,2023-05-25 02:00:00+00:00,88,1
1,2023-06-25 07:00:00+00:00,85,1
2,2023-01-21 05:00:00+00:00,192,0
3,2023-12-12 10:00:00+00:00,221,0
4,2023-10-13 03:00:00+00:00,93,1
...,...,...,...
2452291,2023-12-27 11:00:00+00:00,216,0
2452292,2023-11-13 13:00:00+00:00,234,139
2452293,2023-12-08 00:00:00+00:00,11,0
2452294,2023-07-10 21:00:00+00:00,229,83


In [41]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=2
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-04 07:51:30,174 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 07:51:30,177 INFO: Initializing external client
2025-03-04 07:51:30,177 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 07:51:30,736 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215646
Current UTC time: 2025-03-04 12:51:30.173938+00:00
Next hour: 2025-03-04 13:00:00+00:00
Querying for date range: 2025-03-04 to 2025-03-05
Filtering for hour: 2025-03-04 13:00
Found 0 records


In [43]:
now = datetime.now(timezone.utc)

In [45]:
predictions

,pickup_hour,pickup_location_id,rides


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2452296 entries, 0 to 2452295
Data columns (total 3 columns):
 #   Column              Dtype                  
---  ------              -----                  
 0   pickup_hour         datetime64[us, Etc/UTC]
 1   pickup_location_id  int32                  
 2   rides               int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 37.4 MB


In [49]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [51]:
dt = current_date.ceil('h')


In [55]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=2  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-04 07:51:54,789 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 07:51:54,791 INFO: Initializing external client
2025-03-04 07:51:54,792 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 07:51:55,455 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215646
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.62s) 


In [75]:
import os
# Ensure Arrow Flight is enabled:
os.environ["HOPSWORKS_DISABLE_ARROW_FLIGHT"] = "false"

import hopsworks
import pandas as pd
from hsfs.feature_store import FeatureStore

# Replace with your actual project name and API key
project = hopsworks.login(
    project="NYC_taxi_data_pro",
    api_key_value="NED5z1BDIN3LWVy1.zo5ROYxBl57FgFYTnVyXlFa26OUGFF4lpeaIgvbeiwiCG3Ow1oVUEmgATUXLOcGE"
)

fs = project.get_feature_store()

# Retrieve the feature group (update name and version as needed)
fg = fs.get_feature_group(name="aml_nyc_taxi", version=2)

# Define the filter time range
start_filter = "2025-02-13"
end_filter = "2025-02-14"

# Set read options to force Spark execution and increase timeout
read_opts = {"arrow_flight_config": {"use_spark": True, "timeout": 120000}}

# Attempt to read the data using the specified read options
df = fg.filter((fg.pickup_hour >= start_filter) & (fg.pickup_hour < end_filter)) \
       .read(read_options=read_opts)

print("DataFrame shape:", df.shape)
print(df.head())


2025-03-04 08:05:53,789 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 08:05:53,794 INFO: Initializing external client
2025-03-04 08:05:53,794 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 08:05:54,343 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215646
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.05s) 
DataFrame shape: (6024, 3)
                pickup_hour  pickup_location_id  rides
0 2025-02-13 01:00:00+00:00                 147      0
1 2025-02-13 00:00:00+00:00                 155      0
2 2025-02-13 02:00:00+00:00                 164     11
3 2025-02-13 07:00:00+00:00                  52      2
4 2025-02-13 05:00:00+00:00                  15      0


In [77]:
results

,pickup_hour,pickup_location_id,rides
0,2025-02-20 21:00:00+00:00,210,0
1,2025-03-01 15:00:00+00:00,216,3
2,2025-02-13 01:00:00+00:00,147,0
3,2025-02-23 18:00:00+00:00,217,3
4,2025-02-20 16:00:00+00:00,217,2
...,...,...,...
112945,2025-02-26 13:00:00+00:00,242,1
112946,2025-02-16 16:00:00+00:00,78,1
112947,2025-03-02 20:00:00+00:00,175,0
112948,2025-02-21 17:00:00+00:00,254,2


In [87]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=2)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [89]:
predictions = fetch_next_hour_predictions()

2025-03-04 08:06:26,779 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 08:06:26,781 INFO: Initializing external client
2025-03-04 08:06:26,781 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 08:06:27,458 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215646
Current UTC time: 2025-03-04 13:06:26.779818+00:00
Next hour: 2025-03-04 14:00:00+00:00
Found 0 records


In [93]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=2)



2025-03-04 08:06:37,787 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 08:06:37,790 INFO: Initializing external client
2025-03-04 08:06:37,790 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 08:06:38,388 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215646


In [95]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (8.18s) 


In [96]:
df["pickup_hour"].max()

Timestamp('2025-03-03 18:00:00+0000', tz='Etc/UTC')

In [97]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-04 14:00:00+00:00


In [98]:
df[df["pickup_hour"] == next_hour]

,pickup_hour,pickup_location_id,rides
